In [9]:
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, LpMaximize
import pandas as pd
import matplotlib.pyplot as plt
import nashpy as nash

In [10]:
ge = np.vectorize(lambda x, y: x >= y)
le = np.vectorize(lambda x, y: x <= y)

# Решение симплекс-методом
def strat_mix(M, F_type, C_type_f):
    model = LpProblem("Game", F_type)
    W = []

    for i in range(M.shape[0]):
        w = LpVariable(f'w{i+1}', lowBound=0)
        W.append(w)

    W = np.array(W)
    

    F = (W).sum()

    C = C_type_f((M * W).sum(axis=1), 1)
    
    model += F
    for constraint in C:
        model += constraint

    model.solve()

    opt = np.array(list(map(lambda x: x.value(), model.variables())))

    v = 1 / opt.sum()
    opt = opt / opt.sum()
    return opt, v

### 1. Решение задачи о нахождении выигрышной стратегии для антагонистической игры

In [15]:
strategies_A = ['Телефоны','Ноутбуки']
strategies_B = ['Телефоны', 'Наушники']

# Матрица игры
M = np.array([
    [10, 7],
    [8, 9]])

worst_response_scenarios_A = M.min(axis=1)
worst_response_scenarios_B = M.max(axis=0)
opt_strategy_A = worst_response_scenarios_A.argmax()
opt_strategy_B = worst_response_scenarios_B.argmin()

opt, v = strat_mix(M, LpMaximize, le)

res = f'a) Оптимальная чистая стратегия для игрока А: №{opt_strategy_A + 1} - {strategies_A[opt_strategy_A]}\n' \
  f'б) Цена игры для игрока А при выборе чистой оптимальной стратегии:  {worst_response_scenarios_A[opt_strategy_A]}\n' \
  f'в) Оптимальная чистая стратегия для игрока Б: №{opt_strategy_B + 1} - {strategies_B[opt_strategy_B]}\n' \
  f'г) Цена игры для игрока Б при выборе чистой оптимальной стратегии: {worst_response_scenarios_B[opt_strategy_B]}\n' \
  f'д) Таблица смешанных стратегий для игрока А: {opt}\n' \
  f'е) Цена игры для игрока А при выборе смешанной оптимальной стратегии: {v}'

print(res)

a) Оптимальная чистая стратегия для игрока А: №2 - Ноутбуки
б) Цена игры для игрока А при выборе чистой оптимальной стратегии:  8
в) Оптимальная чистая стратегия для игрока Б: №2 - Наушники
г) Цена игры для игрока Б при выборе чистой оптимальной стратегии: 9
д) Таблица смешанных стратегий для игрока А: [0.5 0.5]
е) Цена игры для игрока А при выборе смешанной оптимальной стратегии: 8.500000059500001
